In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

In [2]:
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [40]:
prompt = ChatPromptTemplate.from_template("what is the capital of {country}")

In [41]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [42]:
chain= prompt| model | StrOutputParser()

In [43]:
population_prompt = ChatPromptTemplate.from_template("What is the population of {capital} ?")

In [44]:
composed_chain = {"capital": chain} | population_prompt | model | StrOutputParser()

In [45]:
composed_chain.invoke({"country": "india"})

'As of 2021, the population of New Delhi, the capital of India, is estimated to be over 21 million.'

#adding memory

In [46]:
from langchain.memory import ConversationBufferWindowMemory

In [47]:
memory = ConversationBufferWindowMemory(k=5)

C:\Windows\Temp\ipykernel_18672\2929478956.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=5)


In [52]:
from langchain.chains import ConversationChain
convo = ConversationChain(
    llm=model,
    memory=memory
)

In [53]:
convo.invoke('what is memory?')

{'input': 'what is memory?',
 'history': '',
 'response': 'Memory is the ability of a system to store, retain, and retrieve information. In the case of a computer system like me, memory refers to the hardware and software components that allow data to be stored temporarily or permanently for future use. There are different types of memory in computers, such as RAM (Random Access Memory) for temporary storage of data while the system is running, and storage devices like hard drives or SSDs for long-term storage of data. Memory plays a crucial role in the functioning of a computer system, as it allows programs to run and data to be accessed quickly and efficiently.'}

#Chatbot with sessio n id and limited memory

In [95]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage

In [96]:
chatbotMemory = {}

# input: session_id, output: chatbotMemory[session_id]
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chatbotMemory:
        chatbotMemory[session_id] = ChatMessageHistory()
    return chatbotMemory[session_id]

In [97]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

In [112]:
def limited_memory_of_messages(messages, number_of_messages_to_keep=5):
    if len(messages) <= number_of_messages_to_keep:
        return messages
    else:
        return messages[-number_of_messages_to_keep:]

In [113]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [114]:
limitedMemoryChain = (
    RunnablePassthrough.assign(messages=lambda x: limited_memory_of_messages(x["messages"]))
    | prompt 
    | model
)

In [115]:
chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limitedMemoryChain,
    get_session_history,
    input_messages_key="messages",
)

In [116]:
session1 = {"configurable": {"session_id": "001"}}

In [117]:
responseFromChatbot = chatbot_with_limited_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my hometown")],
    },
    config=session1,
)

In [118]:
responseFromChatbot.content

'Your hometown is Patna, the capital city of Bihar in India. If you have any more questions or need information about Patna, feel free to ask.'

In [119]:
chatbotMemory

{'001': InMemoryChatMessageHistory(messages=[HumanMessage(content='what is my hometown', additional_kwargs={}, response_metadata={}), AIMessage(content='I apologize, but I do not have access to information about your personal details, including your hometown. If there is any other way I can assist you, please feel free to let me know.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 31, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BJGqtThU9u1g5MF3l31B0aS1hTW34', 'finish_reason': 'stop', 'logprobs': None}, id='run-13a78a1b-a766-4c99-bdbd-0830eb369cfd-0', usage_metadata={'input_tokens': 31, 'output_tokens': 40, 'total_tokens': 71, 'input_token_details': {'audio': 0,